In [1]:
# import necessary modules
import psycopg2 as pg
import pandas as pd
import numpy as np
import time
from datetime import datetime

In [141]:
# try to connect to the server
try:
    # define the attribute of the connection
    conn = pg.connect(
        host = "th-data-test.ckvp0ck3llgr.ap-southeast-1.rds.amazonaws.com",
        port = 5432,
        dbname = "th_data_test",
        user = "applicant",
        password = "Applicant!23"
    )
# if cannot connect, show error
except:
    print("I cannot connect to the database")

In [142]:
# create a dataframe for all distinct members [membership_no datatype character varying, or string]
df_q = "SELECT DISTINCT membership_no FROM public.subscription_data;"
df = pd.read_sql_query(df_q, conn)
# drop all null membership numbers
df.dropna(subset=['membership_no'], how = 'all', inplace = True)
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 987162 entries, 0 to 987161
Data columns (total 1 columns):
membership_no    987162 non-null object
dtypes: object(1)
memory usage: 15.1+ MB
None


,membership_no
0,T-105344298-1
1,T-105344299-1
2,T-105344300-1
3,T-105344301-1
4,T-105344302-1


In [108]:
# find first join date and last free subscription date as TonicBasic
# [first_join_date datatype datetime, last_free_subscription_date datatype datetime]
basic_q = "SELECT membership_no, "\
                "MIN(to_date(subscription_on, 'DD/MM/YYYY')) AS first_join_date, "\
                "MAX(to_date(subscription_on, 'DD/MM/YYYY')) AS last_free_subscription_date "\
            "FROM public.subscription_data "\
            "WHERE current_product_code = 'TonicBasic' "\
            "GROUP BY membership_no;"
basic = pd.read_sql_query(basic_q, conn)
print(basic.info())
basic.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442708 entries, 0 to 442707
Data columns (total 3 columns):
membership_no                  442707 non-null object
first_join_date                442708 non-null object
last_free_subscription_date    442708 non-null object
dtypes: object(3)
memory usage: 10.1+ MB
None


,membership_no,first_join_date,last_free_subscription_date
0,T-105344298-1,2019-01-01,2019-01-01
1,T-105344299-1,2019-01-01,2019-01-01
2,T-105344300-1,2019-01-01,2019-01-01
3,T-105344301-1,2019-01-01,2019-01-01
4,T-105344302-1,2019-01-01,2019-01-01


In [110]:
# First Join date and last subscription date as non-free (non-TonicBasic) 
# [first_join_date datatype datetime, last_free_subscription_date datatype datetime]
nonbasic_q = "SELECT membership_no, "\
                "MIN(to_date(subscription_on, 'DD/MM/YYYY')) AS first_paid_subscription_date, "\
                "MAX(to_date(subscription_on, 'DD/MM/YYYY')) AS last_paid_subscription_date "\
            "FROM public.subscription_data "\
            "WHERE current_product_code NOT IN ('TonicBasic', 'NonMember') "\
            "GROUP BY membership_no;"
nonbasic = pd.read_sql_query(nonbasic_q, conn)
print(nonbasic.info())
nonbasic.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142780 entries, 0 to 142779
Data columns (total 3 columns):
membership_no                   142780 non-null object
first_paid_subscription_date    142780 non-null object
last_paid_subscription_date     142780 non-null object
dtypes: object(3)
memory usage: 3.3+ MB
None


,membership_no,first_paid_subscription_date,last_paid_subscription_date
0,T-105344578-1,2019-01-01,2019-01-01
1,T-105344659-1,2019-01-01,2019-01-01
2,T-105344662-1,2019-01-01,2019-01-01
3,T-105344676-1,2019-01-01,2019-01-01
4,T-105344680-1,2019-01-01,2019-01-01


In [114]:
# First retailer/channel identification 
# [first_retailer datatype character varying, or string
#  first_subscription_channel datatype character varying, or string
#  first_transaction_channel datatype character varying, or string]
q_retail = "SELECT DISTINCT ON (membership_no) "\
            "membership_no, retailer_type AS first_retailer, "\
                "subscription_channel AS first_subscription_channel, "\
                "transaction_channel AS first_transaction_channel "\
            "FROM public.subscription_data "\
            "ORDER BY membership_no, to_date(subscription_on, 'DD/MM/YYYY');"
retail = pd.read_sql_query(q_retail, conn)
print(retail.info())
retail.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 987163 entries, 0 to 987162
Data columns (total 4 columns):
membership_no                 987162 non-null object
first_retailer                55935 non-null object
first_subscription_channel    987163 non-null object
first_transaction_channel     83807 non-null object
dtypes: object(4)
memory usage: 30.1+ MB
None


,membership_no,first_retailer,first_subscription_channel,first_transaction_channel
0,T-105344298-1,None,USSD,None
1,T-105344299-1,None,USSD,None
2,T-105344300-1,None,USSD,None
3,T-105344301-1,None,USSD,None
4,T-105344302-1,None,USSD,None


In [115]:
# last_subscription_status and user_type
# [last_subscription_status datatype character varying, or string
#  user_type datatype character varying, or string]
q_l_sub = "SELECT DISTINCT ON (membership_no) "\
                "membership_no, subscription_status AS last_subscription_status, user_type "\
            "FROM public.subscription_data "\
            "ORDER BY membership_no ASC, to_date(subscription_on, 'DD/MM/YYYY') DESC;"
lsub = pd.read_sql_query(q_l_sub, conn)
print(lsub.info())
lsub.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 987163 entries, 0 to 987162
Data columns (total 3 columns):
membership_no               987162 non-null object
last_subscription_status    987163 non-null object
user_type                   83915 non-null object
dtypes: object(3)
memory usage: 22.6+ MB
None


,membership_no,last_subscription_status,user_type
0,T-105344298-1,Subscribed,None
1,T-105344299-1,Subscribed,None
2,T-105344300-1,Subscribed,None
3,T-105344301-1,Subscribed,None
4,T-105344302-1,Subscribed,None


In [116]:
# Number of paid subscription availed in his lifetime 
# [paid_subscription_count datatype integer]
q_paid = "SELECT membership_no, COUNT(DISTINCT current_product_code ) AS paid_subscription_count "\
            "FROM public.subscription_data "\
            "WHERE current_product_code NOT IN ('TonicBasic', 'NonMember') "\
            "GROUP BY membership_no "\
            "ORDER BY membership_no;"
paid = pd.read_sql_query(q_paid, conn)
print(paid.info())
paid.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142780 entries, 0 to 142779
Data columns (total 2 columns):
membership_no              142780 non-null object
paid_subscription_count    142780 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.2+ MB
None


,membership_no,paid_subscription_count
0,T-105344578-1,1
1,T-105344659-1,1
2,T-105344662-1,1
3,T-105344676-1,1
4,T-105344680-1,1


In [117]:
# Number of renew performed in his lifetime [membership_no datatype character varying, or string]
q_renew = "SELECT membership_no, COUNT(subscription_status) AS renew_count "\
            "FROM public.subscription_data "\
            "WHERE subscription_status = 'RENEW' "\
            "GROUP BY membership_no "\
            "ORDER BY membership_no;"
renew = pd.read_sql_query(q_renew, conn)
print(renew.info())
renew.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35676 entries, 0 to 35675
Data columns (total 2 columns):
membership_no    35676 non-null object
renew_count      35676 non-null int64
dtypes: int64(1), object(1)
memory usage: 557.5+ KB
None


,membership_no,renew_count
0,T-1700000007-1,4
1,T-1700000010-1,4
2,T-1700000013-1,4
3,T-1700000014-1,4
4,T-1700000016-1,4


In [118]:
# average duration of subscription [average_membership_duration_days datatype integer]
q_time = "SELECT membership_no, to_date(subscription_on, 'DD/MM/YYYY') "\
            "FROM public.subscription_data "\
            "WHERE current_product_code = 'NonMember' OR previous_product_code = 'NonMember' "\
            "ORDER BY membership_no, to_date(subscription_on, 'DD/MM/YYYY') ASC;"
stime = pd.read_sql_query(q_time, conn)
print(stime.info())
stime.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1044241 entries, 0 to 1044240
Data columns (total 2 columns):
membership_no    1044224 non-null object
to_date          1044241 non-null object
dtypes: object(2)
memory usage: 15.9+ MB
None


,membership_no,to_date
0,T-105344298-1,2019-01-01
1,T-105344299-1,2019-01-01
2,T-105344300-1,2019-01-01
3,T-105344301-1,2019-01-01
4,T-105344302-1,2019-01-01


In [121]:
# get the subscribe and unsubscribe date of each members as list
times = stime.groupby('membership_no')['to_date'].apply(list)
means = []
for row in times:
    if (len(row) % 2 == 1): # check if odd
        row.append(datetime.now().date()) #append current date
    # get the mean number of days as integer
    mean_diff = np.mean(np.array(row[1::2]) - np.array(row[::2])).days 
    means.append(mean_diff) # append average duration
sub_time = pd.DataFrame({'membership_no':np.array(times.index), 'average_membership_duration_days':np.array(means)}) # convert into dataframe
print(sub_time.head())
sub_time.info()

   average_membership_duration_days  membership_no
0                                67  T-105344298-1
1                                67  T-105344299-1
2                                67  T-105344300-1
3                                67  T-105344301-1
4                                67  T-105344302-1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950192 entries, 0 to 950191
Data columns (total 2 columns):
average_membership_duration_days    950192 non-null int64
membership_no                       950192 non-null object
dtypes: int64(1), object(1)
memory usage: 14.5+ MB


In [122]:
# function to combine all dataframes
def updatedf(col_name, df_name):
    df[col_name] = None
    df.set_index('membership_no', inplace = True)
    df_name.set_index('membership_no', inplace = True)
    df.update(df_name)
    df.reset_index(inplace = True)
    df_name.reset_index(inplace = True)

In [143]:
# average_membership_duration_days
updatedf('average_membership_duration_days', sub_time)
# paid_subscription_count
updatedf('paid_subscription_count', paid)
# renew_count
updatedf('renew_count', renew)
# last_subscription_status
updatedf('last_subscription_status', lsub)
# current subscription user type
updatedf('user_type', lsub)
# first_subscription_channel
updatedf('first_subscription_channel', retail)
# first_transaction_channel
updatedf('first_transaction_channel', retail)
# first_retailer
updatedf('first_retailer', retail)
# last_free_subscription_date
updatedf('last_free_subscription_date', basic)
# last_paid_subscription_date
updatedf('last_paid_subscription_date', nonbasic)
# first_paid_subscription_date
updatedf('first_paid_subscription_date', nonbasic)
# first_join_date
updatedf('first_join_date', basic)
df.head()

,membership_no,average_membership_duration_days,paid_subscription_count,renew_count,last_subscription_status,user_type,first_subscription_channel,first_transaction_channel,first_retailer,last_free_subscription_date,last_paid_subscription_date,first_paid_subscription_date,first_join_date
0,T-105344298-1,67,None,None,Subscribed,None,USSD,None,None,2019-01-01,None,None,2019-01-01
1,T-105344299-1,67,None,None,Subscribed,None,USSD,None,None,2019-01-01,None,None,2019-01-01
2,T-105344300-1,67,None,None,Subscribed,None,USSD,None,None,2019-01-01,None,None,2019-01-01
3,T-105344301-1,67,None,None,Subscribed,None,USSD,None,None,2019-01-01,None,None,2019-01-01
4,T-105344302-1,67,None,None,Subscribed,None,USSD,None,None,2019-01-01,None,None,2019-01-01


In [144]:
# check all the datatypes and row numbers
print(df.dtypes)
print(df.info())

membership_no                       object
average_membership_duration_days    object
paid_subscription_count             object
renew_count                         object
last_subscription_status            object
user_type                           object
first_subscription_channel          object
first_transaction_channel           object
first_retailer                      object
last_free_subscription_date         object
last_paid_subscription_date         object
first_paid_subscription_date        object
first_join_date                     object
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 987162 entries, 0 to 987161
Data columns (total 13 columns):
membership_no                       987162 non-null object
average_membership_duration_days    950192 non-null object
paid_subscription_count             142780 non-null object
renew_count                         35676 non-null object
last_subscription_status            987162 non-null object
user_type             

In [145]:
# close connection to postgresql server
conn.close()

In [146]:
# backup the data in file
df.to_csv("data.csv", index = False)

In [127]:
# try to connect to the redshift database
try:
    # define the attribute of the connection
    redcon = pg.connect(
        host = "th-data-test.ccu6ybiolunl.ap-southeast-1.redshift.amazonaws.com",
        port = 5439,
        dbname = "th_data_test",
        user = "applicant",
        password = "Applicant!23"
    )
# if cannot connect, show error
except:
    print("I cannot connect to the database")

In [149]:
# open a cursor
cursor = redcon.cursor()

In [ ]:
# create the table if it does not exist
q_table = '''CREATE TABLE IF NOT EXISTS facts_table 
(membership_no VARCHAR PRIMARY KEY,
average_membership_duration_days INT,
paid_subscription_count INT,
renew_count INT,
last_subscription_status VARCHAR,
user_type VARCHAR,
first_subscription_channel VARCHAR,
first_transaction_channel VARCHAR,
first_retailer VARCHAR,
last_free_subscription_date DATE,
last_paid_subscription_date DATE,
first_paid_subscription_date DATE,
first_join_date DATE);'''
cursor.execute(q_table)
redcon.commit()

In [156]:
# list the headers of the dataframe
headers = list(df)
# convert the dataframe into tuple
data = list(df.itertuples(index=False, name=None))
# the table you are writing into
table_name = "public.facts_table"
# join column headers with comma
cols = ', '.join(headers)
# as many %s as columns
records = ', '.join(['%s'] * len(headers))
# create the INSERT INTO... statement
insert_str = ("INSERT INTO "+ table_name + " ("+ cols+ ") VALUES ")
print(insert_str)

INSERT INTO public.facts_table (membership_no, average_membership_duration_days, paid_subscription_count, renew_count, last_subscription_status, user_type, first_subscription_channel, first_transaction_channel, first_retailer, last_free_subscription_date, last_paid_subscription_date, first_paid_subscription_date, first_join_date) VALUES 


In [157]:
# cursor.mogrify cannot process all data in the dataframe at once
# so chunk the data and write it faster
# here, each chunk is 100k long, the loop writes first 900000 data
chunk_size = 100000
for i in range(1,10):
    # join data tuples as byte strings with comma
    # the b is for byte strings
    args_str = b','.join(cursor.mogrify("("+records+")", x) for x in data[(i-1)*chunk_size:i*chunk_size])
    # write the data into table
    cursor.execute(insert_str+args_str.decode("utf-8"))
    # commit the change
    redcon.commit()
    print(i)
# write the last portion of data
args_str = b','.join(cursor.mogrify("("+records+")", x) for x in data[i*chunk_size:])
cursor.execute(insert_str+args_str.decode("utf-8"))
redcon.commit()
print("All Done!")

1
2
3
4
5
6
7
8
9
All Done!


In [158]:
# close the cursor
cursor.close()

In [ ]:
# delete all data from the table
# cursor.execute("truncate public.facts_table;")

In [ ]:
# delete the table
# cursor.execute("DROP public.member_subscription_fact;")

In [160]:
# see how many rows got written
view_q = "SELECT COUNT(*) FROM public.facts_table;"
view = pd.read_sql_query(view_q,redcon)
view.head()

,count
0,987162


In [161]:
# see all the data written into the table
view_q = "SELECT * FROM public.facts_table LIMIT 5;"
view = pd.read_sql_query(view_q,redcon)
view.head()

,membership_no,average_membership_duration_days,paid_subscription_count,renew_count,last_subscription_status,user_type,first_subscription_channel,first_transaction_channel,first_retailer,last_free_subscription_date,last_paid_subscription_date,first_paid_subscription_date,first_join_date
0,T-105344299-1,67,None,None,Subscribed,None,USSD,None,None,2019-01-01,None,None,2019-01-01
1,T-105344301-1,67,None,None,Subscribed,None,USSD,None,None,2019-01-01,None,None,2019-01-01
2,T-105344303-1,67,None,None,Subscribed,None,USSD,None,None,2019-01-01,None,None,2019-01-01
3,T-105344305-1,67,None,None,Subscribed,None,USSD,None,None,2019-01-01,None,None,2019-01-01
4,T-105344307-1,67,None,None,Subscribed,None,USSD,None,None,2019-01-01,None,None,2019-01-01


In [162]:
# close cunnection
redcon.close()